In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

In [0]:
device = torch.device('cuda')

In [3]:
train_data = torchvision.datasets.CIFAR10(root = './data', train = True, download = True, transform = transforms.ToTensor())
test_data = torchvision.datasets.CIFAR10(root = './data', train = False, download = True, transform = transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset = train_data, batch_size = 50, shuffle = True)
test_loader = torch.utils.data.DataLoader(dataset = test_data, batch_size = 50, shuffle = False)


Files already downloaded and verified
Files already downloaded and verified


In [0]:
class ConvNN(nn.Module) :
  def __init__ (self) :
    super(ConvNN, self).__init__()
    self.clayer1 = nn.Sequential(
      nn.Conv2d(3, 16, kernel_size = 5, padding = 2, stride = 1),
      nn.BatchNorm2d(16),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.clayer2 = nn.Sequential(
      nn.Conv2d(16, 32, kernel_size = 5, padding = 2, stride = 1),
      nn.BatchNorm2d(32),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size = 2, stride = 2))
    self.dlayer = nn.Sequential(
      nn.Linear(8 * 8 * 32, 512),
      nn.ReLU(),
      nn.Dropout(p = 0.5),
      nn.Linear(512, 128),
      nn.ReLU(),
      nn.Dropout(p = 0.5),
      nn.Linear(128, 10))
  
  def forward(self, x) :
    x = self.clayer1(x)
    x = self.clayer2(x)
    x = x.reshape(x.size(0), -1)
    x = self.dlayer(x)
    return x

In [0]:
learning_rate = 0.0005
max_epochs = 50
# print(len(train_loader))
# print(len(test_loader))

In [0]:
cnn = ConvNN().to(device)

In [0]:
criterion = nn.CrossEntropyLoss()
opt = optim.Adam(cnn.parameters(), lr = learning_rate)

In [15]:
for epoch in range(max_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        opt.zero_grad()
        pred = cnn.forward(images)
        loss = criterion(pred, labels)
        loss.backward()
        opt.step()
        if (i + 1) % 500 == 0:
          print('Epoch : {} / {}  |  Batch : {}  |  Loss : {:.3f}'.format(epoch + 1, max_epochs, i + 1, loss))
          
print("Finished Training")

Epoch : 1 / 50  |  Batch : 500  |  Loss : 1.433
Epoch : 1 / 50  |  Batch : 1000  |  Loss : 1.649
Epoch : 2 / 50  |  Batch : 500  |  Loss : 1.144
Epoch : 2 / 50  |  Batch : 1000  |  Loss : 1.139
Epoch : 3 / 50  |  Batch : 500  |  Loss : 1.171
Epoch : 3 / 50  |  Batch : 1000  |  Loss : 0.870
Epoch : 4 / 50  |  Batch : 500  |  Loss : 1.131
Epoch : 4 / 50  |  Batch : 1000  |  Loss : 1.113
Epoch : 5 / 50  |  Batch : 500  |  Loss : 1.014
Epoch : 5 / 50  |  Batch : 1000  |  Loss : 1.073
Epoch : 6 / 50  |  Batch : 500  |  Loss : 0.659
Epoch : 6 / 50  |  Batch : 1000  |  Loss : 0.689
Epoch : 7 / 50  |  Batch : 500  |  Loss : 0.955
Epoch : 7 / 50  |  Batch : 1000  |  Loss : 0.671
Epoch : 8 / 50  |  Batch : 500  |  Loss : 0.961
Epoch : 8 / 50  |  Batch : 1000  |  Loss : 1.011
Epoch : 9 / 50  |  Batch : 500  |  Loss : 0.718
Epoch : 9 / 50  |  Batch : 1000  |  Loss : 1.211
Epoch : 10 / 50  |  Batch : 500  |  Loss : 0.542
Epoch : 10 / 50  |  Batch : 1000  |  Loss : 0.913
Epoch : 11 / 50  |  Batch : 

In [16]:
cnn.eval()
with torch.no_grad() :
  
  correct = 0
  total = 0
  
  for images, labels in test_loader :
    images = images.to('cuda')
    labels = labels.to('cuda')
    
    out = cnn.forward(images)
    
    pred = torch.argmax(out.data, 1)
    
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  print(correct)
  print(total)
  print(100 * correct / total)


7496
10000
74.96


In [17]:
cnn.eval()
with torch.no_grad() :
  
  correct = 0
  total = 0
  
  for images, labels in train_loader :
    images = images.to('cuda')
    labels = labels.to('cuda')
    
    out = cnn.forward(images)
    
    pred = torch.argmax(out.data, 1)
    
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  print(correct)
  print(total)
  print(100 * correct / total)

49465
50000
98.93
